# Step Size is a Consequential Parameter in Continuous Cellular Automata

Some cells take a significant amount of wall time to execute, which may be a problem for free cloud notebooks like mybinder.org. Experiment with changing the number of simulation steps or CA grid dimensions for shorter execution times. In this notebook, these cells can take a bit longer to run: 
   * Code for building Figure 2 and Figure 3

In [ ]:
# this section needs to be run for setup on colab
# uncomment and run the commands below if on colab
# then restart the runtime and run tests in the next cell


#! git clone https://github.com/riveSunder/yuca.git my_yuca
#%cd my_yuca
#! pip install -e .

In [ ]:
#! python -m testing.test_all

In [ ]:
import yuca
from yuca.multiverse import CA
from yuca.code import CODE
from yuca.patterns import get_orbium
from yuca.utils import seed_all

import torch
import numpy as np

import skimage
import skimage.io as sio
import skimage.transform

import matplotlib

import matplotlib.pyplot as plt
my_cmap = plt.get_cmap("magma")
plt.rcParams['figure.dpi'] = 300

import IPython

import yuca.analyze

from yuca.zoo.librarian import Librarian

torch.set_default_dtype(torch.float32)

#plotting utilities:
sad = lambda x, y: (x-y).abs().mean()

def get_center_of_mass(grid, spacing=20):
    
    if len(grid.shape) > 2:
        my_grid = np.array(grid.cpu().squeeze())
    else:
        my_grid = np.array(1.0 * grid.cpu())
        
    edge_l = my_grid[:,:spacing].sum()
    edge_r = my_grid[:,-spacing:].sum()
    edge_t = my_grid[:spacing,:].sum()
    edge_b = my_grid[-spacing:,:].sum()
    
    max_edge = np.max([edge_l, edge_r, edge_t, edge_b])
    
    xx_grid, yy_grid = np.meshgrid(np.arange(my_grid.shape[-1]), np.arange(my_grid.shape[-2]))
    
    if max_edge:
        
        if edge_r == max_edge:
            com_x = xx_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            com_y = yy_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            
        elif edge_l == max_edge:
            com_x = xx_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            com_y = yy_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            
        elif edge_t == max_edge:
            com_x = xx_grid[:-spacing, :] * my_grid[:-spacing, :] / my_grid[:-spacing, :].sum()
            com_y = yy_grid[:-spacing, :] * my_grid[:-spacing, :] /my_grid[:-spacing, :].sum()
            
        else: # edge_b == max_edge:
            com_x = xx_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            com_y = yy_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            
    else:
        # pattern is not at an edge crossing
        
        com_x = (xx_grid * my_grid) / my_grid.sum()
        com_y = (yy_grid * my_grid) / my_grid.sum()
    
    return com_x.sum(), com_y.sum()


def plot_grids(slow_grid, grid, fast_grid, my_cmap=plt.get_cmap("magma"), titles=["Step size demo"], vmin=0.0, vmax=1):

    global subplot_0
    global subplot_1
    global subplot_2

    if titles == None:
        titles = ["Title!"]

    fig, ax = plt.subplots(1,3, figsize=(4.5,1.5), facecolor="white")

    # TODO invert cmap
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0 = ax[0].imshow(slow_grid_display, interpolation="nearest")
    subplot_1 = ax[1].imshow(grid_display, interpolation="nearest")
    subplot_2 = ax[2].imshow(fast_grid_display, interpolation="nearest")
    
    fig.suptitle(titles[0], fontsize=8)
    for hh in range(3):
        ax[hh].set_yticklabels('')
        ax[hh].set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global subplot_1    
    global subplot_2
    global slow_grid
    global grid
    global fast_grid
    global ax
    global speedup_factor
    
    dt = 1.0 *ca.dt
    
    for ii in range(speedup_factor):
        grid = ca(grid)

        ca.dt = dt / 3.
        slow_grid = ca(slow_grid)
        ca.dt = min([dt * 3, 1.0])
        fast_grid = ca(fast_grid)
        ca.dt = dt
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0.set_array(slow_grid_display)
    subplot_1.set_array(grid_display)
    subplot_2.set_array(fast_grid_display)
    
    ax[0].set_title(f"Step size: {dt/3:.4f}", fontsize=7)
    ax[1].set_title(f"Step size: {dt:.4f}", fontsize=7)
    ax[2].set_title(f"Step size: {dt*3:.4f}", fontsize=7)
    
    plt.tight_layout()


def plot_compare(grid_0, grid_1, grid_2, grid_3, my_cmap=plt.get_cmap("magma"),invert=True, titles=None):

    global subplot_0
    global subplot_1
    global subplot_2
    global subplot_3

    if titles == None:
        titles = [f"Grid {count}" for count in range(4)]
        
    fig, ax = plt.subplots(2,2, figsize=(3.15, 2.57), gridspec_kw={'width_ratios': [1,1]})
    
    my_cmap = plt.get_cmap(my_cmap)
    
    if grid_0.shape[-1] != 3:
        if invert:
            grid_0_display = 1.0 - my_cmap(grid_0.cpu().squeeze())[:,:,:3]
            grid_1_display = 1.0 - my_cmap(grid_1.cpu().squeeze())[:,:,:3]
            grid_2_display = 1.0 - my_cmap(grid_2.cpu().squeeze())[:,:,:3]
            grid_3_display = 1.0 - my_cmap(grid_3.cpu().squeeze())[:,:,:3]
        else:
            grid_0_display = my_cmap(grid_0.cpu().squeeze())[:,:,:3]
            grid_1_display = my_cmap(grid_1.cpu().squeeze())[:,:,:3]
            grid_0_display = my_cmap(grid_0.cpu().squeeze())[:,:,:3]
            grid_2_display = my_cmap(grid_2.cpu().squeeze())[:,:,:3]
            grid_3_display = my_cmap(grid_3.cpu().squeeze())[:,:,:3]
    else:
        grid_0_display = grid_0
        grid_1_display = grid_1
        grid_2_display = grid_2
        grid_3_display = grid_3
        
    subplot_0 = ax[0,0].imshow(grid_0_display, interpolation="nearest")                      
    ax[0,0].set_ylabel(titles[0], fontsize=5.5)
        
    subplot_1 = ax[0,1].imshow(grid_1_display, interpolation="nearest")                          
    ax[0,1].set_ylabel(titles[1], fontsize=5.5)
    
    subplot_2 = ax[1,0].imshow(grid_2_display, interpolation="nearest")                      
    ax[1,0].set_ylabel(titles[2], fontsize=5.5)
        
    subplot_3 = ax[1,1].imshow(grid_3_display, interpolation="nearest")                          
    ax[1,1].set_ylabel(titles[3], fontsize=5.5)
                           
    ax[0,0].set_xticklabels("")
    ax[0,0].set_yticklabels("")
    ax[0,1].set_xticklabels("")
    ax[0,1].set_yticklabels("")
    ax[1,0].set_xticklabels("")
    ax[1,0].set_yticklabels("")
    ax[1,1].set_xticklabels("")
    ax[1,1].set_yticklabels("")
    
    plt.suptitle("Different Step Size, Different Behavior", fontsize=7.)
                    
    plt.tight_layout()

    return fig, ax

def update(i):
    global grid_0
    global grid_1
    global grid_2
    global grid_3
    global ax
        
    old_grid_0 = 1.0 * grid_0
    old_grid_1 = 1.0 * grid_1
    old_grid_2 = 1.0 * grid_2
    old_grid_3 = 1.0 * grid_3
    
    accumulated_t = 1.0 * ca_0.t_count
    
    
    while ca_0.t_count < (accumulated_t + max([0.1, ca_0.dt]) ):
        grid_0 = ca_0(grid_0)
        
    while ca_1.t_count < (accumulated_t + max([0.1, ca_0.dt]) ):
        grid_1 = ca_1(grid_1)
        
    while ca_2.t_count < (accumulated_t + max([0.1, ca_0.dt]) ):
        grid_2 = ca_2(grid_2)
        
    while ca_3.t_count < (accumulated_t + max([0.1, ca_0.dt]) ):
        grid_3 = ca_3(grid_3)
        
    if 1: #invert:
        grid_0_display = 1.0 - my_cmap(grid_0.cpu().squeeze())[:,:,:3]
        grid_1_display = 1.0 - my_cmap(grid_1.cpu().squeeze())[:,:,:3]
        grid_2_display = 1.0 - my_cmap(grid_2.cpu().squeeze())[:,:,:3]
        grid_3_display = 1.0 - my_cmap(grid_3.cpu().squeeze())[:,:,:3]
    else:
        grid_0_display = my_cmap(grid_0.cpu().squeeze())[:,:,:3]
        grid_1_display = my_cmap(grid_1.cpu().squeeze())[:,:,:3]
        grid_2_display = my_cmap(grid_2.cpu().squeeze())[:,:,:3]
        grid_3_display = my_cmap(grid_3.cpu().squeeze())[:,:,:3]
        
    subplot_0.set_array(grid_0_display)
    subplot_1.set_array(grid_1_display)
    subplot_2.set_array(grid_2_display)
    subplot_3.set_array(grid_3_display)
    
    ax[0,0].set_title(f"{ca_0.dt} step size \n accumulated time: {ca_0.t_count}")
    ax[0,1].set_title(f"{ca_1.dt} step size \n accumulated time: {ca_1.t_count}")
    ax[1,0].set_title(f"{ca_2.dt} step size \n accumulated time: {ca_2.t_count}")
    ax[1,1].set_title(f"{ca_3.dt} step size \n accumulated time: {ca_3.t_count}")
    
   


## Interactive Demo

Choose a pattern and step size to explore how $dt$ affects pattern stability and behavior

In [ ]:
num_frames = 96
speedup_factor = 2

#instantiate librarian
lib = Librarian()
lib.update_index()

print("Available patterns:")

print("Index      pattern name")
for idx in range(len(lib.index)):
    print(f"{idx}      {lib.index[idx]}")

try:
    pattern_index = min([len(lib.index), int(input(prompt="\nChoose a pattern index:"))])
except:
    pattern_index = 1
    
try:
    step_size = float(input(prompt="\nChoose a step size (0.001 to 1.0):"))
    step_size = np.clip(step_size, 0.001, 1.0)
except:
    step_size = .1
    
try:
    num_frames = int(input(prompt="\nNumber of frames to animate? (max 256): "))
    num_frames = min([max([1, num_frames]), 256])
except:
    num_frames = 1
    
try:
    speedup_factor = int(input(prompt="\nAnimations with small step size can be slow. Choose a speedup factor (max 100, 1 to display every frame): "))
    speedup_factor = max([1, speedup_factor])
    speedup_factor = min([100, speedup_factor])
except:
    speedup_factor = 1
    
print("\n")

pattern_name = lib.index[pattern_index]

pattern, ca_meta = lib.load(pattern_name)

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.dt = step_size

dim_x = max([128, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()

slow_grid = 1.0 * grid
fast_grid = 1.0 * grid 
fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])

plt.close("all")
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=100).to_jshtml())

## Continue simulating current grid (optional)

In [ ]:
try:
    step_size = float(input(prompt="\nChoose a step size (0.001 to 1.0):"))
    step_size = np.clip(step_size, 0.001, 1.0)
except:
    step_size = .1

try:
    num_frames = int(input(prompt="\nNumber of frames to animate? (max 256): "))
    num_frames = min([max([1, num_frames]), 256])
except:
    num_frames = 1
    
try:
    speedup_factor = int(input(prompt="\nAnimations with small step size can be slow. Choose a speedup factor (max 100, 1 to display every frame): "))
    speedup_factor = max([1, speedup_factor])
    speedup_factor = min([100, speedup_factor])
except:
    speedup_factor = 1
    
    
    
ca.dt = step_size

print("\n \n    (note that you'll need to run the next cell to generate an animation)")

In [ ]:

fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])

plt.close("all")
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=100).to_jshtml())

## Figure 1: Stability of a minimal glider in Lenia's _Scutium gravidus_ rule set

In [ ]:
pattern_name = "scutium_gravidus_single000"
pattern, ca_meta = lib.load(pattern_name)
ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

dim = 64

number_steps = 64
max_t = number_steps / 0.97

my_device = "cpu"

my_grid = np.zeros((dim, int(dim*3.5)))

my_grid[-2*pattern.shape[-2]:-pattern.shape[-2], :pattern.shape[-1]] = pattern#[0,0]

grid = torch.tensor(my_grid).reshape(1,1,my_grid.shape[0], my_grid.shape[1])
grid = grid.to(torch.get_default_dtype()).to(my_device)

ca_978 = CA(device=my_device)
ca_978.restore_config(ca_config)
ca_978.no_grad()
ca_978.to_device(my_device)

ca_978.t_count = 0.0
ca_978.dt = 0.978

ca_970 = CA(device=my_device)
ca_970.restore_config(ca_config)
ca_970.no_grad()
ca_970.to_device(my_device)

ca_970.t_count = 0.0
ca_970.dt = 0.970

ca_245 = CA(device=my_device)
ca_245.restore_config(ca_config)
ca_245.no_grad()
ca_245.to_device(my_device)

ca_245.t_count = 0.0
ca_245.dt = 0.245

com_978 = []
com_970 = []
com_245 = []

last_grid_978 = None
last_grid_950 = None
last_grid_245 = None

grid_978 = 1.0 * grid
grid_970 = 1.0 * grid
grid_245 = 1.0 * grid

my_green = 1.0 - np.array(my_cmap(grid.cpu().max().numpy()))[:3]
my_blue = 1.0 - np.array(plt.get_cmap("afmhot")(grid.cpu().max().numpy()))[:3]
 
while ca_978.t_count < max_t:
    grid_978 = ca_978(grid_978)



    last_grid_978 = 1.0 * grid_978
    if grid_978.mean() == 0.0:
        print(f"{ca_978.dt} grid gone at t_count = {ca_978.t_count}")
        break

    com_978.append(get_center_of_mass(grid_978, spacing=14))


while ca_970.t_count < max_t:
    grid_970 = ca_970(grid_970)

    last_grid_970 = 1.0 * grid_970
    if grid_970.mean() == 0.0:
        print(f"{ca_970.dt} grid gone at t_count = {ca_970.t_count}")
        break

    com_970.append(get_center_of_mass(grid_970, spacing=14))


while ca_245.t_count < max_t:
    grid_245 = ca_245(grid_245)

    last_grid_245 = 1.0 * grid_245
    if grid_245.mean() == 0.0:
        print(f"{ca_245.dt} grid gone at t_count = {ca_245.t_count}")
        break

    com_245.append(get_center_of_mass(grid_245, spacing=14))


grid_0_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_0_display = 1.0 - (grid_0_display + my_cmap(grid_978.squeeze().cpu())[:,:,:3])

grid_1_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_1_display = 1.0 - (grid_1_display + my_cmap(grid_970.squeeze().cpu())[:,:,:3])

grid_2_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_2_display = 1.0 - (grid_2_display + my_cmap(grid_245.squeeze().cpu())[:,:,:3])

fig, ax = plt.subplots(3, 1, figsize=(3.5, 2.5))


ax[0].imshow(grid_0_display, interpolation="nearest")
ax[1].imshow(grid_1_display, interpolation="nearest")
ax[2].imshow(grid_2_display, interpolation="nearest")


for ii in range(len(com_978)):
    progress = ii / len(com_978)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[0].scatter(com_978[ii][0], com_978[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[0].set_ylabel(f"Unstable \nstep size {ca_978.dt}", fontsize=5.5)

for ii in range(len(com_970)):
    progress = ii / len(com_970)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[1].scatter(com_970[ii][0], com_970[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[1].set_ylabel(f"Stable \nstep size {ca_970.dt}", fontsize=5.5)

for ii in range(len(com_245)):
    progress = ii / len(com_245)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[2].scatter(com_245[ii][0], com_245[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[2].set_ylabel(f"Unstable \nstep size {ca_245.dt}", fontsize=5.5)

for xx in range(3):
    ax[xx].set_xticklabels("")
    ax[xx].set_yticklabels("")

    
for note, (ii), (xx, yy) in [("start", (0), (3,50)), ("", (0), (8,5)),\
                               ("start", (1), (3,50)), ("end", (1), (160,45)), \
                               ("start", (2), (3,50)), ("", (2), (8,3))]:

    ax[ii].annotate(note, xy= (xx, yy), fontsize=5)
    
plt.suptitle("Figure 1: Scutium gravidus single glider", fontsize=7)
plt.tight_layout()
plt.show()

plt.close("all")




## Figure 2: Stability of a wide glider in _Scutium gravidus_

Note that if executing this notebook with mybinder, it's likely to timeout before finishing the simulation for this figure. Try setting `max_t` to a lower value, _e.g._ 10, though the pattern won't become unstable in too short of a simulation.


In [ ]:
pattern_name = "scutium_gravidus_superwide_000"
pattern, ca_meta = lib.load(pattern_name)
ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

dim = 200

max_t = 100

# more manageable on mybinder
#max_t = 32

seed_all(42)

my_device = "cpu"

my_grid = np.zeros((dim, int(dim*3.5)))

my_grid[-10-pattern.shape[-2]:-10, :pattern.shape[-1]] = pattern#[0,0]

grid = torch.tensor(my_grid).reshape(1,1,my_grid.shape[0], my_grid.shape[1])
grid = grid.to(torch.get_default_dtype()).to(my_device)

ca_978 = CA(device=my_device)
ca_978.restore_config(ca_config)
ca_978.no_grad()
ca_978.to_device(my_device)

ca_978.t_count = 0.0
ca_978.dt = 0.5

ca_970 = CA(device=my_device)
ca_970.restore_config(ca_config)
ca_970.no_grad()
ca_970.to_device(my_device)

ca_970.t_count = 0.0
ca_970.dt = 0.1

ca_245 = CA(device=my_device)
ca_245.restore_config(ca_config)
ca_245.no_grad()
ca_245.to_device(my_device)

ca_245.t_count = 0.0
ca_245.dt = 0.05

com_978 = []
com_970 = []
com_245 = []

last_grid_978 = None
last_grid_950 = None
last_grid_245 = None

grid_978 = 1.0 * grid
grid_970 = 1.0 * grid
grid_245 = 1.0 * grid

my_green = 1.0 - np.array(my_cmap(grid.cpu().max().numpy()))[:3]
my_blue = 1.0 - np.array(plt.get_cmap("afmhot")(grid.cpu().max().numpy()))[:3]
 
while ca_978.t_count < max_t:
    grid_978 = ca_978(grid_978)

    last_grid_978 = 1.0 * grid_978
    if grid_978.mean() == 0.0:
        print(f"{ca_978.dt} grid gone at t_count = {ca_978.t_count}")
        break

    com_978.append(get_center_of_mass(grid_978, spacing=14))


while ca_970.t_count < max_t:
    grid_970 = ca_970(grid_970)

    last_grid_970 = 1.0 * grid_970
    if grid_970.mean() == 0.0:
        print(f"{ca_970.dt} grid gone at t_count = {ca_970.t_count}")
        break

    com_970.append(get_center_of_mass(grid_970, spacing=14))


while ca_245.t_count < max_t:
    grid_245 = ca_245(grid_245)

    last_grid_245 = 1.0 * grid_245
    if grid_245.mean() == 0.0:
        print(f"{ca_245.dt} grid gone at t_count = {ca_245.t_count}")
        break

    com_245.append(get_center_of_mass(grid_245, spacing=14))


grid_0_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_0_display = 1.0 - (grid_0_display + my_cmap(grid_978.squeeze().cpu())[:,:,:3])

grid_1_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_1_display = 1.0 - (grid_1_display + my_cmap(grid_970.squeeze().cpu())[:,:,:3])

grid_2_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_2_display = 1.0 - (grid_2_display + my_cmap(grid_245.squeeze().cpu())[:,:,:3])

fig, ax = plt.subplots(3, 1, figsize=(3.5, 2.5))


ax[0].imshow(grid_0_display, interpolation="nearest")
ax[1].imshow(grid_1_display, interpolation="nearest")
ax[2].imshow(grid_2_display, interpolation="nearest")


for ii in range(len(com_978)):
    progress = ii / len(com_978)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[0].scatter(com_978[ii][0], com_978[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[0].set_ylabel(f"Unstable \nstep size {ca_978.dt}", fontsize=5.5)

for ii in range(len(com_970)):
    progress = ii / len(com_970)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[1].scatter(com_970[ii][0], com_970[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[1].set_ylabel(f"Stable \nstep size {ca_970.dt}", fontsize=5.5)

for ii in range(len(com_245)):
    progress = ii / len(com_245)
    my_color = progress * my_green + (1.0 - progress) * my_blue

    ax[2].scatter(com_245[ii][0], com_245[ii][1], color=my_color, s=1.0, alpha=0.125) 

    ax[2].set_ylabel(f"Unstable \nstep size {ca_245.dt}", fontsize=5.5)

for xx in range(3):
    ax[xx].set_xticklabels("")
    ax[xx].set_yticklabels("")
    


for note, (ii), (xx, yy) in [("start", (0), (3,50)), ("", (0), (8,5)),\
                               ("start", (1), (3,50)), ("end", (1), (380,45)), \
                               ("start", (2), (3,50)), ("", (2), (8,3))]:

    ax[ii].annotate(note, xy= (xx, yy), fontsize=5)

plt.suptitle("Figure 2: Scutium gravidus wide glider", fontsize=7)
plt.tight_layout()
plt.show()

plt.close("all")




## Figure 3: Different step size leads to qualitatively different behavior
This simulation is likely to timeout if you're running this notebook with mybinder. Try setting `num_frames` to a lower value, such as 48. 

In [ ]:
# static figure for paper
num_frames = 512
# try few frames if running on mybinder
# num_frames = 48

dim = 128
stop_at_t = 13.0*2.5+1.2

pattern_name = "frog000"
my_device = "cpu"
pattern, ca_meta = lib.load(pattern_name)
ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca_0 = CA(device=my_device)
ca_0.restore_config(ca_config)

ca_1 = CA(device=my_device)
ca_1.restore_config(ca_config)

ca_2 = CA(device=my_device)
ca_2.restore_config(ca_config)

ca_3 = CA(device=my_device)
ca_3.restore_config(ca_config)


my_grid = np.zeros((dim, dim))
my_grid[dim -pattern.shape[-2]:dim + pattern.shape[-2], :pattern.shape[-1]] = pattern[0,0]

grid = torch.tensor(my_grid).reshape(1,1,my_grid.shape[0], my_grid.shape[1])
grid = grid.to(torch.get_default_dtype()).to(my_device)

ca_0.reset()
ca_1.reset()
ca_2.reset()
ca_3.reset()

ca_0.to(my_device)
ca_1.to(my_device)
ca_2.to(my_device)
ca_3.to(my_device)

ca_0.no_grad()
ca_1.no_grad()
ca_2.no_grad()
ca_3.no_grad()

ca_0.dt = 0.13
ca_1.dt = 0.1
ca_2.dt = 0.025
ca_3.dt = 0.0125

grid_0 = 1.0 * grid
grid_1 = 1.0 * grid
grid_2 = 1.0 * grid
grid_3 = 1.0 * grid

com_0 = []
com_1 = []
com_2 = []
com_3 = []

while ca_0.t_count < stop_at_t: # - ca_0.dt:
    grid_0 = ca_0(grid_0)
    com_0.append(get_center_of_mass(grid_0))
    
while ca_1.t_count < stop_at_t: #- ca_1.dt:
    grid_1 = ca_1(grid_1)
    com_1.append(get_center_of_mass(grid_1))
    
while ca_2.t_count < stop_at_t: #- ca_2.dt:
    grid_2 = ca_2(grid_2)
    
    com_2.append(get_center_of_mass(grid_2))
    
while ca_3.t_count < stop_at_t: #- ca_3.dt:
    grid_3 = ca_3(grid_3)
    com_3.append(get_center_of_mass(grid_3))

my_titles = []
for ca in [ca_0, ca_1, ca_2, ca_3]:
    my_titles.append(f"accumulated $t = ${ca.t_count:.2f} \n step size $dt = ${ca.dt:.4f}")

grid_0_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_0_display = 1.0 - (grid_0_display + my_cmap(grid_0.squeeze().cpu())[:,:,:3])

grid_1_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_1_display = 1.0 - (grid_1_display + my_cmap(grid_1.squeeze().cpu())[:,:,:3])

grid_2_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_2_display = 1.0 - (grid_2_display + my_cmap(grid_2.squeeze().cpu())[:,:,:3])

grid_3_display = plt.get_cmap("afmhot")(grid.squeeze().cpu())[:,:,:3]
grid_3_display = 1.0 - (grid_3_display + my_cmap(grid_3.squeeze().cpu())[:,:,:3])

my_titles = []
for ca in [ca_0, ca_1, ca_2, ca_3]:
    my_titles.append(f"accumulated $t = ${ca.t_count:.2f} \n step size $dt = ${ca.dt:.3f}")
    
fig, ax = plot_compare(grid_0_display, grid_1_display, grid_2_display, grid_3_display, titles=my_titles, my_cmap=my_cmap, invert=True)


my_green = 1.0 - np.array(my_cmap(grid.cpu().max().numpy()))[:3]
my_blue = 1.0 - np.array(plt.get_cmap("afmhot")(grid.cpu().max().numpy()))[:3]

if(1):
    for ii in range(len(com_0)):
        progress = ii / len(com_0)
        my_color = progress * my_green + (1.0 - progress) * my_blue

        ax[0,0].scatter(com_0[ii][0],com_0[ii][1], color=my_color, s=0.5, alpha=0.05) 

    for ii in range(len(com_1)):
        progress = ii / len(com_1)
        my_color = progress * my_green + (1.0 - progress) * my_blue

        ax[0,1].scatter(com_1[ii][0],com_1[ii][1], color=my_color, s=0.5, alpha=0.05) 

    for ii in range(len(com_2)):
        progress = ii / len(com_2)
        my_color = progress * my_green + (1.0 - progress) * my_blue

        ax[1,0].scatter(com_2[ii][0],com_2[ii][1], color=my_color, s=0.5, alpha=0.05) 

    for ii in range(len(com_3)):
        progress = ii / len(com_3)
        my_color = progress * my_green + (1.0 - progress) * my_blue

        ax[1,1].scatter(com_3[ii][0],com_3[ii][1], color=my_color, s=0.5, alpha=0.05) 

    
for note, (ii,jj), (xx, yy) in [("start", (0,0), (30,120)), ("end", (0,0), (85,35)),\
                               ("start", (0,1), (30,120)), ("end", (0,1), (95,80)), \
                               ("start", (1,0), (30,120)), ("end", (1,0), (75,10)),\
                               ("start", (1,1), (30,120)), ("end", (1,1), (85,30))]:
    
    ax[ii,jj].annotate(note, xy= (xx, yy), fontsize=5)

plt.tight_layout()
#plt.savefig("assets/frog_trajectories.png")
plt.show()
